In [ ]:
import tensorflow as tf 
tf.test.gpu_device_name()


''

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/DNLP_project/code/KRED-master'

/content/drive/.shortcut-targets-by-id/1hnC14IVjSyGsivvXdzusFlK9UiEdnsuj/DNLP_project/code/KRED-master


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=3383d178c423cb0626c5cbd8f28f0ab3b4b78ae58c17e6f2c430bad8804b3d93
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


# Introduction

This repository is the implementation of [KRED: Knowledge-Aware Document Representation for News Recommendations](https://arxiv.org/abs/1910.11494) [1]


## Model description



KRED is a knowledge enhanced framework which enhance a document embedding with knowledge information for multiple news recommendation tasks. The framework mainly contains two part: representation enhancement part(left) and multi-task training part(right).

![](./framework.PNG)

## Dataset description and download

MIND dataset [2] is a large-scale English news dataset. It was collected from anonymized behavior logs of Microsoft News website. MIND contains 1,000,000 users, 161,013 news articles and 15,777,377 impression logs. Every news article contains rich textual content including title, abstract, body, category and entities. Each impression log contains the click events, non-clicked events and historical news click behaviors of this user before this impression.

For quicker training and evaluaiton, we sample MINDdemo dataset of 5k users from MIND small dataset. The MINDdemo dataset has the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall and MINDlarge, please change the dowload source. Select the MIND_type parameter from ['large', 'small', 'demo'] to choose dataset.

MINDdemo_train is used for training, and MINDdemo_dev is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

In [ ]:
import os
from utils.util import *
from train_test import *

In [ ]:
import os
from utils.util import *
from train_test import *

# # Options: demo, small, large
# MIND_type = 'demo'
# data_path = "./data/"

# train_news_file = os.path.join(data_path, 'train', r'news.tsv')
# train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
# valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
# valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
# knowledge_graph_file = os.path.join(data_path, 'kg/wikidata-graph', r'wikidata-graph.tsv')
# entity_embedding_file = os.path.join(data_path, 'kg/wikidata-graph', r'entity2vecd100.vec')
# relation_embedding_file = os.path.join(data_path, 'kg/wikidata-graph', r'relation2vecd100.vec')

# mind_url, mind_train_dataset, mind_dev_dataset, _ = get_mind_data_set(MIND_type)

# kg_url = "https://kredkg.blob.core.windows.net/wikidatakg/"

# if not os.path.exists(train_news_file):
#     download_deeprec_resources(mind_url, \
#                                os.path.join(data_path, 'train'), mind_train_dataset)
    
# if not os.path.exists(valid_news_file):
#     download_deeprec_resources(mind_url, \
#                                os.path.join(data_path, 'valid'), mind_dev_dataset)

# if not os.path.exists(knowledge_graph_file):
#     download_deeprec_resources(kg_url, \
#                                os.path.join(data_path, 'kg'), "kg.zip")

## loading config

In [ ]:
import sys
import os
sys.path.append('')

import argparse
from parse_config import ConfigParser


parser = argparse.ArgumentParser(description='KRED')


parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default='all', type=str,
                    help='indices of GPUs to enable (default: all)')

config = ConfigParser.from_args(parser)


try to use this

In [ ]:
import yaml

fp_yaml =  open('./config.yaml')
config = yaml.safe_load(fp_yaml)

In [ ]:
config['trainer']

{'training_type': 'single_task',
 'task': 'item2item',
 'epochs': 5,
 'smooth_lamda': 10,
 'train_neg_num': 4,
 'save_dir': './out/saved/',
 'save_period': 3,
 'early_stop': 2,
 'monitor': 'val_auc',
 'verbosity': 1}

## Create hyper-parameters

In [ ]:
epochs = 5
batch_size = 64
train_type = "single_task"
task = "user2item" # task should be within: user2item, item2item, vert_classify, pop_predict

config['trainer']['epochs'] = epochs
config['data_loader']['batch_size'] = batch_size
config['trainer']['training_type'] = train_type
config['trainer']['task'] = task

## Process dataset

Since MIND dataset do not contain user's location information, we can not use local news 


In [ ]:
data = load_data_mind(config)

constructing adjacency matrix ...
creating dictionary of graph


100%|██████████| 31963632/31963632 [00:49<00:00, 648375.82it/s]


adding entity to id relationship


100%|██████████| 3275149/3275149 [01:34<00:00, 34672.08it/s]


start build entity
start train news
start valid news


...

73152it [00:01, 37800.68it/s]


constructing embedding ...
entity embeddings extraction


3275149it [01:58, 27562.35it/s]


relation embeddings extraction


1091it [00:01, 818.28it/s]


In [ ]:
import pickle
file_name = 'data_mind_small_i2i_ver2.pkl'
f = open(f"/content/drive/MyDrive/DNLP_project/code/KRED-master/{file_name}", 'wb')
pickle.dump(data, f)
f.close()

In [ ]:
train = data[-2]

In [ ]:
for i,k in enumerate(train['item2']):
  if k == '':
    print(i)


10326
49191
59439
66191
119461
142727
150984
166896
186261
188859


In [ ]:
from itertools import islice
del train['item2'][10326]

In [ ]:
del train['item1'][10326]
del train['label'][10326]

In [ ]:
l = []
for i,k in enumerate(train['item2']):
  if k == '':
    del train['item2'][i]
    del train['item1'][i]
    del train['label'][i]


In [ ]:
for i,k in enumerate(train['item2']):
  if k == '':
    print(i)

In [ ]:
test = data[-1]

In [ ]:
for i,k in enumerate(test['item1']):
  if k == '':
    print(i)

## Train the KRED model

In [ ]:
train_type = "single_task"

if train_type == "single_task":
    single_task_training(config, data)
else:
    multi_task_training(config, data)

RuntimeError: ignored

## Evaluate the KRED model

In [ ]:
test_data = data[-1]
testing(test_data, config)

## Performance on MINDlarge

we test the performance on MINDlarge dev dataset for your reference:

| Models | AUC | NDCG@10 |
| :------- | :------- | :------- |
| KRED(single task training) | 0.6702 | 0.4018 |
| KRED(multi task training) |  0.6731 | 0.4039|


## Reference

[1] Liu, Danyang, et al. "KRED: Knowledge-Aware Document Representation for News Recommendations." Fourteenth ACM Conference on Recommender Systems. 2020.

[2] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics.

In [4]:
n = 10
f = open('/content/drive/MyDrive/DNLP_project/code/KRED-master/data/kg/wikidata-graph/description.txt', 'r')
for i in range(n):
  print(f.readline().strip())

Q31	federal constitutional monarchy in Western Europe
Q8	mental or emotional state of well-being characterized by pleasant emotions
Q23	First President of the United States
Q24	character from the television series 24
Q42	English writer and humorist
Q1868	Belgian author and visionary
Q2013	free knowledge database project hosted by Wikimedia and edited by volunteers
Q45	republic in Southwestern Europe
Q51	polar continent
Q58	primary sexual organ that male animals use to inseminate sexually receptive mates (females or hermaphrodites) during copulation, occurring in many (but not all) animals, including humans
